In [44]:
# import h5 reader
import h5py
import numpy as np
import pandas as pd
from os import listdir
from os.path import isfile, join

work_path = "/home/shengduo/pylith-developer/build/debug/pylith-nonRegSlipLawWithVaryingB/examples/bar_shearwave/quad4/"
h5_path = work_path + "output/fault/"

In [64]:
# Class RunABatch
class RunABatch:
    # Constructor
    def __init__(self, input_set, work_path, FourierTerms = 16):
        # Initialize data paths and batch parameters
        self.work_path = work_path
        self.input_set = input_set
        self.h5_path = work_path + "output/fault/"
        self.FourierTerms = FourierTerms
        
        # Store all cases in the h5_path
        self.existingCasesFile = self.h5_path + "CaseList.csv"
        
        # Flag for whether cases have been run
        self.casesExcuted = False
        
        # Get existing cases
        self.getExistingCasesOfInputSet()
        
        # Get input_set_toRun
        self.input_set_toRun = list(set(input_set) - self.existingCases)
        
        # Run cases
        self.runCases(self.input_set_toRun)
        
        # Get obsevations
        self.Observations = self.getObservations(self.input_set)
    
    # Inline function gets [A, B] list
    def getAAndB(self, fileName):
        A_idx = fileName.find('A')
        B_idx = fileName.find('B')
        slash_idx = fileName.find('-')
        
        # Change this part !! Before applying to new name convention
        A = float(fileName[A_idx + 1 : B_idx - 1])
        B = float(fileName[B_idx + 1 : slash_idx])
        return (A, B)
    
    
    # Function get_existing_files_set
    def getExistingCasesOfInputSet(self):
        # Get all .h5 file names as a list
        myPath = self.h5_path
        onlyFiles = [f for f in listdir(myPath) if (isfile(join(myPath, f)) and f[-3 : ] == '.h5')]
        self.existingCases = set([self.getAAndB(f) for f in onlyFiles])
        
    # Function runCases
    def runCases(self, input_set):
        shell_path = self.work_path + "RunJobsJP.sh"
        shellRead = open(shell_path, 'r')
        list_of_lines = shellRead.readlines()
        shellRead.close()

        AA = [ele[0] for ele in input_set]
        BB = [ele[1] for ele in input_set]

        list_of_lines[9] = "AA=" + str(tuple(AA)).replace(',', '') + "\n"
        list_of_lines[10] = "BB=" + str(tuple(BB)).replace(',', '') + "\n"

        shellWrite = open(shell_path, 'w')
        shellWrite.writelines(list_of_lines)
        shellWrite.close()

        # Run the cases
        !source $shell_path
        
        self.casesExcuted = True
        
        # Return from shell
        return
    
    
    # Function getObservations for the input_set
    def getObservations(self, input_set):
        # Initialize Observations
        Observations = []
        
        # Check if the cases have been excuted
        if self.casesExcuted == False:
            return Observations
        
        # Loop through all Inputs
        for input_ele in input_set:
            # Open the file
            h5_file = self.h5_path + "A" + str(input_ele[0]) + "_B" + str(input_ele[1]) + "-fault.h5"
            f = h5py.File(h5_file, 'r')

            # Get time
            time = np.array(f['time']).reshape([-1])
            time = time - np.min(time)
            nOfTSteps = time.shape[0]

            # Get Slip rates
            SlipRates = np.array(f['vertex_fields']['slip_rate'])
            Vx = SlipRates[:, :, 0].transpose()
            Vy = SlipRates[:, :, 1].transpose()
            nOfNodes = Vx.shape[0]

            # Find the Fourier coefficients
            FourierTerms = self.FourierTerms
            T = np.max(time)

            # Compute the Fourier terms
            Ks = np.array(range(FourierTerms))
            coskPiTt = np.cos(Ks.reshape([-1, 1]) * np.pi / T * time)
            VxcoskPiTt = np.concatenate([coskPiTt * Vxi.reshape([1, -1]) for Vxi in Vx], 0)

            # Compute the fourier coefficients
            # print('time.shape: ', time.shape)
            observation = np.trapz(VxcoskPiTt, x=time)

            # Append the result from this file
            Observations.append(observation)
            
        return Observations
    
    

In [66]:
# Input example of getting observations
AA = np.linspace(0.002, 0.014, 7)
BB = AA + 0.004

# Get input set
input_set = set([(AA[i], BB[i]) for i in range(len(AA))])

# Test class RunABatch
myBatch = RunABatch(input_set, work_path)

Running case A0.014_B0.018000000000000002
Finished in 12 s!

/home/shengduo/InverseProblems/GPRWorkingField


# Sampling process

In [106]:
# Import the sampling packages
import emcee

# Set up a GP
myGP = GP_predictor(myBatch.input_set, myBatch.Observations)

# Define the log probability function
def log_prob(u, y, GP, si_eta = 1.):
    return - 0.5 / si_eta ** 2 * np.sum((y - GP.predict(u)) ** 2)

# Use a sample y
y = myBatch.Observations[0]

# Sample from p(u|y)
uSt = np.array([0.002, 0.006])
uLength = np.array([0.01, 0.01])

ndim, nwalkers = 2, 100
p0 = np.random.uniform(size = [nwalkers, ndim]) * uLength + uSt

sampler = emcee.EnsembleSampler(nwalkers, ndim, log_prob, args=[y, myGp])
sampler.run_mcmc(p0, 1000)

ModuleNotFoundError: No module named 'emcee'

In [ ]:
a = np.random.uniform()

# Gaussian predictor

In [72]:
# Import Gaussian-regression related functions
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF

# Pre-process the data
from sklearn import preprocessing

# function train_GP
class GP_predictor:
    # Constructor
    def __init__(self, 
                 input_set, 
                 observation_set, 
                 GPkernel = 1 * RBF(length_scale=1.0, length_scale_bounds=(1e-1, 1e2)), 
                 n_optimizers = 9):
        # Scale input data
        self.input_scaler = preprocessing.StandardScaler()
        self.input_scaler.fit(np.array(input_set))
        
        # Scale output data
        self.observation_scaler = preprocessing.StandardScaler()
        self.observation_scaler.fit(np.array(observation_set))
        
        # Fit Gaussian process
        self.GP = GaussianProcessRegressor(kernel = GPkernel, n_restarts_optimizer = n_optimizers)
        self.GP.fit(self.input_scaler.transform(np.array(input_set)), 
                    self.observation_scaler.transform(np.array(observation_set)))
        
    # Predict on a new input set
    def predict(self, new_input_set):
        # Predict new observation
        new_observation = self.observation_scaler.inverse_transform(
            self.GP.predict(
                self.input_scaler.transform(np.array(new_input_set))
            )
        )
        
        return new_observation
        
    

In [69]:
# Generate a set, run cases, get observations

In [ ]:
myGP = GP_predictor(input_set, )